# Script to analyse MSIT fMRI data through nilearn

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import nibabel as nib
import nilearn
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
import re
import os
import numpy as np
import pandas as pd
import pickle as pkl
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import multiprocessing as mp
from functools import partial
import itertools

import copy
from utils import apply_warp

%matplotlib inline

In [ ]:
from datetime import datetime
tuples_to_run = [(x.split('/')[-2],x.split('/')[-1]) for x in sorted(glob.glob('../sourcedata/zipdata/sub*/ses*'))]
tuples_to_run = [x for x in tuples_to_run if not x[0] == 'sub-001']

for sub,ses in tuples_to_run:
    glob.glob(f'../derivatives2/fmriprep/fmriprep/{sub}/{ses}/func/*')

## Model specifications
### Model 0:
- Con
- Simon
- Flanker
- Inc

### Model 1:
- Response_index
- Response_middle
- Response_ring

In [ ]:
def make_first_level_model_fn(sub, ses, task, smoothing_fwhm, model_n, space, save_dir_name='../derivatives/glm_nilearn/subject_level_model'):
    smoothing_fwhm = str(smoothing_fwhm).replace('.', 'p')
    return os.path.join(save_dir_name, f'sub-{sub}/ses-{ses}/func/fwhm-{smoothing_fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-first-level-model.pkl')

## load
def load_first_level_model(fn):
    if os.path.exists(fn):
        with open(fn, 'rb') as f:
            first_level_model = pkl.load(f)  # protocol=4 required due to size of pkl
        return first_level_model
    else:
        print(f'{fn} does not exist...')
        return 0
    
def fit_first_level_model(sub, ses, task, model_n=0, space='MNI152NLin2009cAsym', include_physio=True, n_jobs=20, smoothing_fwhm=4.5, 
                          use_susan_presmoothed_data=True,
                          save_model=True, save_dir_name='subject_level_model',
                          overwrite=False, return_model=True):
    # check for existing model
    smoothing_fwhm_str = str(smoothing_fwhm).replace('.', 'p')
    save_fn = make_first_level_model_fn(sub, ses, task, smoothing_fwhm, model_n, space, save_dir_name) # f'../derivatives/{save_dir_name}/sub-{sub}/ses-{ses}/func/fwhm-{smoothing_fwhm_str}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-first-level-model.pkl'
    if os.path.exists(save_fn) and not overwrite:
        print(f'Model {model_n} {sub} {ses} {task} {smoothing_fwhm} already fit! Returning pre-fit model...')
        if return_model:
            return load_first_level_model(save_fn)
        else:
            return 0
    
    event_fns = sorted(glob.glob(f'../derivatives/event_files/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-*_events.tsv'))
#     regressor_fns = sorted(glob.glob(f'../derivatives/behavior/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-*_desc-model-regressors.tsv'))
    confounds_fns = sorted(glob.glob(f'../derivatives/fmriprep/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-*_desc-confounds_timeseries.tsv'))
    
    # fix this for susan presmoothed data
    if use_susan_presmoothed_data:
        smoothing_fwhm=None
        func_fns = sorted(glob.glob(f'../derivatives/susan_smoothed_func/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-*_space-{space}_desc-preproc_bold_smoothed_fwhm-{smoothing_fwhm_str}.nii.gz'))
    else:
        func_fns = sorted(glob.glob(f'../derivatives/fmriprep/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-*_space-{space}_desc-preproc_bold.nii.gz'))
        
    run_flms = []
    run_events = []
    run_funcs = []
    run_confounds = []
    
    for run, (event_fn,  func_fn, confounds_fn) in enumerate(zip(event_fns, func_fns, confounds_fns)):    
        # Merge events with trial-by-trial regressors
        events = pd.read_csv(event_fn, sep='\t', index_col=None)  # Onsets & event types
        events['duration'] = 0.001  # stick functions
        if model_n == 0:
            events = events.loc[events.trial_type.isin(['con', 'simon', 'flanker', 'inc'])]
        elif model_n == 1:
            events = events.loc[events.trial_type.isin(['response_index', 'response_middle', 'response_ring'])]
        events = events[['onset', 'trial_type', 'duration']]
        
        run_events.append(events)

        # load func
        func = nib.load(func_fn)
        run_funcs.append(func)

        # get confounds
        confounds = pd.read_csv(confounds_fn, sep='\t')[['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'dvars', 'framewise_displacement']].fillna(method='bfill')
        
        # get retroicor
        if include_physio:
            run_idx = run+1
            retroicor_fn = f'../derivatives/retroicor/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-{run_idx}_desc-retroicor_regressors.tsv'
            if not os.path.exists(retroicor_fn):
                ## take first 20 aCompCor components
                print("No retroicor found, including 20 a_comp_cor components")
                a_comp_cor = pd.read_csv(confounds_fn, sep='\t')[['a_comp_cor_' + str(x).zfill(2) for x in range(20)]]
                confounds = pd.concat([confounds, a_comp_cor], axis=1)
            else:
                retroicor = pd.read_csv(retroicor_fn, sep='\t', header=None).iloc[:,:20]  ## 20 components in total
                retroicor.columns = ['cardiac_' + str(x) for x in range(6)] + ['respiratory_' + str(x) for x in range(8)] + ['respiratoryxcardiac_' + str(x) for x in range(4)] + ['HRV', 'RVT']
                confounds = pd.concat([confounds, retroicor], axis=1)

        run_confounds.append(confounds)
   
    
    # get brain mask
    brain_mask = f'../derivatives/fmriprep/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-1_space-{space}_desc-brain_mask.nii.gz'
    
    # fit
    flm = FirstLevelModel(t_r=1.38, hrf_model = 'glover + derivative', slice_time_ref=0.5,
                          mask_img=brain_mask, smoothing_fwhm=smoothing_fwhm,
                          n_jobs=n_jobs, subject_label=sub)
    flm_fitted = flm.fit(run_funcs, events=run_events, confounds=run_confounds)
    
    # save
    if save_model:
        print('saving model to {}'.format(save_fn))
        os.makedirs(os.path.dirname(save_fn), exist_ok=True)
        with open(save_fn, 'wb') as f:
            pkl.dump(flm_fitted, f, protocol=4)  # protocol=4 required due to size of pkl
    
    if return_model:
        return flm_fitted
    else:
        return 0

# Which participants?

In [ ]:
reg = re.compile('.*sub-(?P<sub>\d+)_ses-sstmsit_task-msit_run-(?P<run>\d).*')

#all_func_files = sorted(glob.glob('../derivatives/fmriprep/fmriprep/sub-*/ses-rlsat/func/sub-*_ses-rlsat_task-rlsat_run-*_space-T1w_desc-preproc_bold.nii.gz'))
all_func_files = sorted(glob.glob('../derivatives/fmriprep/fmriprep/sub-*/ses-sstmsit/func/sub-*_ses-sstmsit_task-msit_run-*_space-T1w_desc-preproc_bold.nii.gz'))
func_file_df = pd.DataFrame([{**reg.match(x).groupdict(), **{'Func file': True}} for x in all_func_files])

all_smoothed_func_files = sorted(glob.glob('../derivatives/susan_smoothed_func/sub-*/ses-sstmsit/func/*_task-msit*'))
susan_file_df = pd.DataFrame([{**reg.match(x).groupdict(), **{'Susan smoothed': True}} for x in all_smoothed_func_files])

all_retroicor_files = sorted(glob.glob('../derivatives/retroicor/sub-*/ses-sstmsit/func/sub-*_ses-sstmsit_task-msit_run-*_desc-retroicor_regressors.tsv'))
retroicor_file_df = pd.DataFrame([{**reg.match(x).groupdict(), **{'Retroicor': True}} for x in all_retroicor_files])

all_event_files = sorted(glob.glob('../derivatives/event_files/sub-*/ses-sstmsit/func/sub-*_ses-sstmsit_task-msit_run-*_events.tsv'))
event_file_df = pd.DataFrame([{**reg.match(x).groupdict(), **{'Events': True}} for x in all_event_files])

# all_behavior_files = sorted(glob.glob('../derivatives/behavior/sub-*/ses-sstmsit/func/sub-*_ses-rlsat_task-rlsat*'))
# behav_file_df = pd.DataFrame([{**reg.match(x).groupdict(), **{'Behavior model': True}} for x in all_behavior_files])

combined_df = pd.merge(pd.merge(pd.merge(func_file_df, susan_file_df, on=['sub', 'run'], how='outer'), 
                                         retroicor_file_df,on=['sub', 'run'], how='outer'), 
                                event_file_df, on=['sub', 'run'], how='outer')
combined_df = combined_df.set_index(['sub', 'run']).fillna(False)
combined_df['Complete'] = combined_df.apply(np.sum, axis=1) == 4

with pd.option_context('display.max_rows', 10000):
    display(combined_df.sort_index())

In [ ]:
to_fit = [(x, 'sstmsit', 'msit') for x in combined_df.loc[combined_df['Func file']].reset_index()['sub'].unique()]
to_fit

# fit it

In [ ]:
# Fit # watch out for overwrite 
save_dir_name = '../derivatives/glm_nilearn/subject_level_model'
space = 'T1w'

def fit_glm(sub,ses,task, model_n, smoothing_fwhm, space='T1w'):
    print(sub)
    out = fit_first_level_model(sub, ses, task, model_n=model_n, space=space, overwrite=False, use_susan_presmoothed_data=True, smoothing_fwhm=smoothing_fwhm, save_dir_name=save_dir_name, return_model=False)
    return out

for model_n in [0, 1]: 
    for smoothing_fwhm in [4.5, 1.5]:
        _ = joblib.Parallel(n_jobs=1)(joblib.delayed(fit_glm)(sub, ses, task, model_n=model_n, smoothing_fwhm=smoothing_fwhm) for sub,ses,task in to_fit)

Compute & warp the contrasts

In [ ]:
def compute_contrasts(comb, contrast_mapping, model_n, smoothing_fwhm, overwrite=False, space='T1w', save_dir_name='../derivatives/glm_nilearn/subject_level_model'):
    sub,ses,task = comb
    # check for existing model
    smoothing_fwhm_str = str(smoothing_fwhm).replace('.', 'p')
    flm_fn = make_first_level_model_fn(sub, ses, task, smoothing_fwhm, model_n, space, save_dir_name=save_dir_name)
    if not os.path.exists(flm_fn):
        raise(IOError(f"Model doesn''t exist! expected to find it at {flm_fn}"))
        
    # check for existing output fns
    stat_map_name = 'z_score'
    to_compute = {} #copy.copy(contrast_mapping)
    for contrast_name, contrast in contrast_mapping.items():
        save_fn = flm_fn.replace('desc-first-level-model.pkl', f'desc-contrast-{contrast_name}_{stat_map_name}.nii.gz')

        if not os.path.exists(save_fn):
            # skip this one
            to_compute[contrast_name] = contrast

        if os.path.exists(save_fn) and overwrite:
            to_compute[contrast_name] = contrast
    
    if len(to_compute) == 0:
        print(f'All contrasts for sub {sub} already computed')
        return 0
        
    # load first level models & make contrast maps
    first_level_model = load_first_level_model(flm_fn)

    for contrast_name, contrast in to_compute.items():
        
        # save all stat maps
        stat_maps = first_level_model.compute_contrast(contrast, output_type='all')
        for stat_map_name, stat_map in stat_maps.items():
            save_fn = flm_fn.replace('desc-first-level-model.pkl', f'desc-contrast-{contrast_name}_{stat_map_name}.nii.gz')
            stat_map.to_filename(save_fn)
            
            # warp
            stat_map_warped = apply_warp(save_fn, sub)            
            # move
            os.rename(stat_map_warped, save_fn.replace('space-T1w', 'space-MNI152NLin2009cAsym'))
    
    return 0


save_dir_name = '../derivatives/glm_nilearn/subject_level_model'
space = 'T1w'

## weirdly this doesn't work with joblib, so move to multiprocess
for model_n in [0, 1]: #,2]:
    
    if model_n == 0:
#         contrast_mapping = {'con': 'fs',
#                             'sml': 'ss',
#                             'fll': 'go',
#                             'fs-go': 'fs-go',
#                             'ss-go': 'ss-go',
#                             'fs-ss': 'fs-ss',
#                             }
        
        contrast_mapping = {
                            'Inc': 'inc',
                            'Flanker': 'flanker',
                            'Simon': 'simon',
                            'Con': 'con',
                            'Inc-Con': 'inc - con',
                            'Inc-Flanker': 'inc - flanker',
                            'Inc-Simon': 'inc - simon',
                            'Flanker-Con': 'flanker - con',
                            'Flanker-Simon': 'flanker - simon',
                            'Simon-Con': 'simon - con'
                            }
        
    elif model_n == 1:
        contrast_mapping = {'Middle': 'response_middle',
                            'Index': 'response_index',
                            'Ring': 'response_ring',
                            'Middle-Ring': 'response_middle - response_ring',
                            'Middle-Index': 'response_middle - response_index',
                            'Ring-Index': 'response_ring - response_index'
                           }
                            
#         contrast_mapping = {'response_left': 'response_left',
#                             'response_right': 'response_right',
#                             'left-right': 'response_left - response_right'
#                            }
        
    for smoothing_fwhm in [4.5, 1.5]:
        with mp.Pool(8) as p:
            p.map(partial(compute_contrasts, contrast_mapping=contrast_mapping, model_n=model_n, smoothing_fwhm=smoothing_fwhm, overwrite=False), to_fit)

        # for comb in to_fit:        

#             compute_contrasts(comb, contrast_mapping=contrast_mapping, model_n=model_n, smoothing_fwhm=smoothing_fwhm)

# plot individual sub contrasts

## first plot each contrast per sub

In [ ]:
save_dir_name = 'subject_level_model'

space = 'T1w'
susan = True
# fwhm = '4p5' # or '1p5'

# plot each contrast per sub
# code is pretty slow cos it loads the t1w image
# for fwhm in ['1p5','4p5']:
#     for model_n in ['0','1']:
        
#         if model_n == '0':
#             contrast_mapping = {
#                                 'Inc': 'inc',
#                                 'Flanker': 'flanker',
#                                 'Simon': 'simon',
#                                 'Con': 'con',
#                                 'Inc-Con': 'inc - con',
#                                 'Inc-Flanker': 'inc - flanker',
#                                 'Inc-Simon': 'inc - simon',
#                                 'Flanker-Con': 'flanker - con',
#                                 'Flanker-Simon': 'flanker - simon',
#                                 'Simon-Con': 'simon - con'
#                                 }

#         elif model_n == '1':
#             contrast_mapping = {
#                                 'Middle': 'response_middle',
#                                 'Index': 'response_index',
#                                 'Ring': 'response_ring',
#                                 'Middle-Ring': 'response_middle - response_ring',
#                                 'Middle-Index': 'response_middle - response_index',
#                                 'Ring-Index': 'response_ring - response_index'
#                                }

def contrast_per_sub(comb, contrast_mapping, model_n, smoothing_fwhm, space, overwrite=False):
    sub, ses, task = comb
#     for sub, ses, task in to_fit:

    if os.path.exists(f'../figures/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}') and not overwrite:
        print(f'pdf for {sub} {ses} {fwhm} {model_n} already made, skipping..')
        #continue 

    if not os.path.exists(f'../figures/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}') or overwrite:
        os.makedirs(f'../figures/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/', exist_ok=True)

        print(f'plotting sub {sub}')
        f, ax = plt.subplots(len(contrast_mapping.items()),3, figsize=(20, 5*len(contrast_mapping.items())), gridspec_kw={'hspace': 0, 'wspace': 0})

        for counter, (contrast_name, contrast) in enumerate(contrast_mapping.items()):

            if contrast in ['response_middle - response_ring','response_middle - response_index', 'response_ring - response_index']:
                x,y,z = -33,-12,43
            else:
                x,y,z = 9,-16,-2

            if not space == 'T1w':
                t1w = 'MNI152TEMPLATE'
            else:
                t1w = f'/home/Public/trondheim/derivatives/fmriprep/fmriprep/sub-{sub}/anat/sub-{sub}_desc-preproc_T1w.nii.gz'

    #                 if susan:
    #                     z_map = f'../derivatives/glm_nilearn/{save_dir_name}/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_susan_desc-contrast-{contrast_name}_z_score.nii.gz'
    #                 else:
    #                     z_map = f'../derivatives/glm_nilearn/{save_dir_name}/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-contrast-{contrast_name}_z_score.nii.gz'

            z_map = f'../derivatives/glm_nilearn/{save_dir_name}/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-contrast-{contrast_name}_z_score.nii.gz'

            plotting.plot_stat_map(z_map, threshold=2.3, title=f'sub - {sub}, contrast - {contrast_name}', display_mode='x', cut_coords=[x], axes=ax[counter,0], colorbar=False, bg_img=t1w)
            plotting.plot_stat_map(z_map, threshold=2.3,  display_mode='y', cut_coords=[y], axes=ax[counter,1], colorbar=False, bg_img=t1w)
            plotting.plot_stat_map(z_map, threshold=2.3,  display_mode='z', cut_coords=[z],axes=ax[counter,2], bg_img=t1w)


    #             if susan:
    #                 plt.gcf().savefig(f'../figures/sub-{sub}/ses-{ses}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_susan_allContrasts-z_maps.pdf')
    #             else:

        plt.gcf().savefig(f'../figures/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}-allContrasts-z_maps.pdf')
    
    return 0

        
for model_n in ['0','1']:

    if model_n == '0':
        contrast_mapping = {
                            'Inc': 'inc',
                            'Flanker': 'flanker',
                            'Simon': 'simon',
                            'Con': 'con',
                            'Inc-Con': 'inc - con',
                            'Inc-Flanker': 'inc - flanker',
                            'Inc-Simon': 'inc - simon',
                            'Flanker-Con': 'flanker - con',
                            'Flanker-Simon': 'flanker - simon',
                            'Simon-Con': 'simon - con'
                            }

    elif model_n == '1':
        contrast_mapping = {
                            'Middle': 'response_middle',
                            'Index': 'response_index',
                            'Ring': 'response_ring',
                            'Middle-Ring': 'response_middle - response_ring',
                            'Middle-Index': 'response_middle - response_index',
                            'Ring-Index': 'response_ring - response_index'
                           }
        
    for fwhm in ['1p5','4p5']:    
        with mp.Pool(8) as p:
            #print(contrast_mapping)
            p.map(partial(contrast_per_sub, contrast_mapping=contrast_mapping, model_n=model_n, smoothing_fwhm=fwhm, space=space, overwrite=False), to_fit)
            
print('Done')

## Second plot each sub per contrast

In [ ]:
# plot each sub per contrast
# code is pretty slow cos it loads the t1w image
# for susan in [True, False]:
for fwhm in ['1p5','4p5']:
    for model_n in ['0','1']:
        
        if model_n == '0':
            contrast_mapping = {
                                'Inc': 'inc',
                                'Flanker': 'flanker',
                                'Simon': 'simon',
                                'Con': 'con',
                                'Inc-Con': 'inc - con',
                                'Inc-Flanker': 'inc - flanker',
                                'Inc-Simon': 'inc - simon',
                                'Flanker-Con': 'flanker - con',
                                'Flanker-Simon': 'flanker - simon',
                                'Simon-Con': 'simon - con'
                                }

        elif model_n == '1':
            contrast_mapping = {
                                'Middle': 'response_middle',
                                'Index': 'response_index',
                                'Ring': 'response_ring',
                                'Middle-Ring': 'response_middle - response_ring',
                                'Middle-Index': 'response_middle - response_index',
                                'Ring-Index': 'response_ring - response_index'
                               }
        
        for contrast_name, contrast in contrast_mapping.items():
            print(f'plotting contrast {contrast_name}')
            f, ax = plt.subplots(len(to_fit),3, figsize=(20, 5*len(to_fit)), gridspec_kw={'hspace': 0, 'wspace': 0})
            for counter, (sub, ses, task) in enumerate(to_fit):

                if contrast in ['response_middle - response_ring','response_middle - response_index', 'response_ring - response_index']:
                    x,y,z = -33,-12,43
                else:
                    x,y,z = 9,5,-2

                if not space == 'T1w':
                    t1w = 'MNI152TEMPLATE'
                else:
                    t1w = f'/home/Public/trondheim/derivatives/fmriprep/fmriprep/sub-{sub}/anat/sub-{sub}_desc-preproc_T1w.nii.gz'

    #             if susan:
    #                 z_map = f'../derivatives/{save_dir_name}/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_space-{space}_susan_desc-contrast-{contrast_name}_z_score.nii.gz'
    #             else:
    #                 z_map = f'../derivatives/{save_dir_name}/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-contrast-{contrast_name}_z_score.nii.gz'
                z_map = f'../derivatives/glm_nilearn/{save_dir_name}/sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-{sub}_ses-{ses}_task-{task}_space-{space}_desc-contrast-{contrast_name}_z_score.nii.gz'

                plotting.plot_stat_map(z_map, threshold=2.3, title=f'Contrast - {contrast_name}, sub - {sub}', display_mode='x', cut_coords=[x], axes=ax[counter,0], colorbar=False, bg_img=t1w)
                plotting.plot_stat_map(z_map, threshold=2.3,  display_mode='y', cut_coords=[y], axes=ax[counter,1], colorbar=False, bg_img=t1w)
                plotting.plot_stat_map(z_map, threshold=2.3,  display_mode='z', cut_coords=[z],axes=ax[counter,2], bg_img=t1w)

            if not os.path.exists(f'../figures/sub-all/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}'):
                os.makedirs(f'../figures/sub-all/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/')
    #         if susan:
    #             plt.gcf().savefig(f'../figures/sub-all/ses-{ses}/sub-ind_ses-{ses}_task-{task}_space-{space}_susan_{contrast_name}-z_maps.pdf', dpi=300,orientation='landscape')
    #         else:
    #             plt.gcf().savefig(f'../figures/sub-all/ses-{ses}/sub-ind_ses-{ses}_task-{task}_space-{space}-{contrast_name}-z_maps.pdf', dpi=300,orientation='landscape')

            plt.gcf().savefig(f'../figures/sub-all/ses-{ses}/func/fwhm-{fwhm}/model-{model_n}/sub-ind_ses-{ses}_task-{task}_space-{space}-{contrast_name}-z_maps.pdf', dpi=300,orientation='landscape')

# Group level

In [ ]:
def save_second_level_spm(spm, first_level_contrast_name, second_level_contrast_name, fwhm, model_n, save_dir_name='../derivatives/glm_nilearn/group_level_model/ses-sstmsit'):
    fwhm = str(fwhm).replace('.', 'p')
    save_fn = os.path.join(save_dir_name, f'fwhm-{fwhm}/model-{model_n}/firstlevelcontrast-{first_level_contrast_name}_secondlevelcontrast-{second_level_contrast_name}.nii.gz')
    
    if not os.path.exists(os.path.dirname(save_fn)):
        os.makedirs(os.path.dirname(save_fn))
    
    spm.to_filename(save_fn)
    nilearn.image.math_img('-nii', nii=spm).to_filename(save_fn.replace('.nii.gz', '-neg.nii.gz'))

In [ ]:
def fit_second_level_models(first_level_contrast, second_level_contrast, fwhm, model_n, df):
    print(f'{fwhm} {model_n} {first_level_contrast}')
    df_in = df.loc[(df.map_name==first_level_contrast) & (df.fwhm==str(fwhm).replace('.', 'p')) & (df.model_n==str(model_n))]
    #dm = pd.merge(df_in, parameters[['subject_label', 'B0.SPD-ACC_z']]) #, 'V0.SPD-ACC_z']])
    df_in['intercept'] = 1

    dm_in = df_in[['intercept']]
#     if first_level_contrast == 'SPD-ACC':
#         dm_in = dm[['intercept', 'B0.SPD-ACC_z']] #, 'V0.SPD-ACC_z']]
#     else:
#         dm_in = dm[['intercept']]
    slm = SecondLevelModel()
    slm_fitted = slm.fit(second_level_input=df_in.effects_map_path.values.tolist(), design_matrix=dm_in)

    # Intercept, threshold covariance, urgency covariance
    cmap = slm_fitted.compute_contrast('intercept', output_type='z_score')
    save_second_level_spm(cmap, first_level_contrast_name=first_level_contrast, second_level_contrast_name=second_level_contrast, fwhm=fwhm, model_n=model_n)

In [ ]:
# fwhm = 1.5
# fwhm_str = str(fwhm).replace('.', 'p')
model_n = '0'

imgs = sorted(glob.glob(f'../derivatives/glm_nilearn/subject_level_model/sub-*/ses-sstmsit/func/fwhm-*/model-{model_n}/*task-msit*MNI*desc-contrast-*_effect_size*'))
regex = re.compile('.*/sub-(?P<sub>\d+)/.*/fwhm-(?P<fwhm>\S+)/model-(?P<model_n>\d)/sub-.*_desc-contrast-(?P<contrast_name>\S+)_effect_size.nii.gz')

df = pd.DataFrame({'effects_map_path':imgs})
df['subject_label'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['sub'])
df['fwhm'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['fwhm'])
df['model_n'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['model_n'])
df['map_name'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['contrast_name'])
df.head()

In [ ]:
all_combs = list(itertools.product(df.map_name.unique().tolist(), ['intercept'], [1.5], [0]))


In [ ]:
import itertools
all_combs = list(itertools.product(df.map_name.unique().tolist(), ['intercept'], [1.5], [0]))

_ = joblib.Parallel(n_jobs=10, verbose=1)(joblib.delayed(fit_second_level_models)(first_level_contrast, second_level_contrast, fwhm, model_n, df=df) for first_level_contrast,second_level_contrast,fwhm,model_n in all_combs)

In [ ]:
all_combs

# Model 0

In [ ]:
mni09c = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
                            'Inc': 'inc',
                            'Flanker': 'flanker',
                            'Simon': 'simon',
                            'Con': 'con',
                            'Inc-Con': 'inc - con',
                            'Inc-Flanker': 'inc - flanker',
                            'Inc-Simon': 'inc - simon',
                            'Flanker-Con': 'flanker - con',
                            'Flanker-Simon': 'flanker - simon',
                            'Simon-Con': 'simon - con'

In [ ]:
for i in [3,0,2,1,4,5,6,7,8,9]:
    first_level_contrast, second_level_contrast, fwhm, _ = all_combs[i]
    fwhm = str(fwhm).replace('.', 'p')
    z_map = nib.load(f'../derivatives/glm_nilearn/group_level_model/ses-sstmsit/fwhm-{fwhm}/model-0/firstlevelcontrast-{first_level_contrast}_secondlevelcontrast-{second_level_contrast}.nii.gz')
    plotting.plot_stat_map(z_map, threshold=3.1, title=first_level_contrast, bg_img=mni09c, cut_coords=[-6, -13, 1], draw_cross=False)
    plt.gcf().set_size_inches(15,6)
    # plt.gcf().savefig(f'./msit-{first_level_contrast}.pdf', bbox_inches='tight')

# Model 1

In [ ]:
# fwhm = 1.5
# fwhm_str = str(fwhm).replace('.', 'p')
model_n = '1'

imgs = sorted(glob.glob(f'../derivatives/glm_nilearn/subject_level_model/sub-*/ses-sstmsit/func/fwhm-*/model-{model_n}/*task-msit*MNI*desc-contrast-*_effect_size*'))
regex = re.compile('.*/sub-(?P<sub>\d+)/.*/fwhm-(?P<fwhm>\S+)/model-(?P<model_n>\d)/sub-.*_desc-contrast-(?P<contrast_name>\S+)_effect_size.nii.gz')

df = pd.DataFrame({'effects_map_path':imgs})
df['subject_label'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['sub'])
df['fwhm'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['fwhm'])
df['model_n'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['model_n'])
df['map_name'] = df.effects_map_path.apply(lambda x: regex.match(x).groupdict()['contrast_name'])
df.head()

In [ ]:
import itertools
all_combs = list(itertools.product(df.map_name.unique().tolist(), ['intercept'], [1.5], [1]))

_ = joblib.Parallel(n_jobs=10, verbose=1)(joblib.delayed(fit_second_level_models)(first_level_contrast, second_level_contrast, fwhm, model_n, df=df) for first_level_contrast,second_level_contrast,fwhm,model_n in all_combs)

In [ ]:
all_combs

In [ ]:
for i in range(6):
    first_level_contrast, second_level_contrast, fwhm, _ = all_combs[i]
    fwhm = str(fwhm).replace('.', 'p')
    z_map = nib.load(f'../derivatives/glm_nilearn/group_level_model/ses-sstmsit/fwhm-{fwhm}/model-1/firstlevelcontrast-{first_level_contrast}_secondlevelcontrast-{second_level_contrast}.nii.gz')
    plotting.plot_stat_map(z_map, threshold=3.1, title=first_level_contrast, bg_img=mni09c, cut_coords=[-6, -13, 1], draw_cross=False)
    plt.gcf().set_size_inches(15,6)
    # plt.gcf().savefig(f'./msit-{first_level_contrast}.pdf', bbox_inches='tight')